In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [2]:
# initialize dataframe with test data
df_train = pd.read_csv('train.csv')

df_train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


## Initial examination:
* Dataset is outdated by 11 years
* Houses are sold between 2006 - 2010 (during housing crash)

In [3]:
# initial exploration
avg_lot_size = df_train['LotArea'].mean()
newest_home_age = 2021 - df_train['YearBuilt'].max()

avg_lot_size, newest_home_age

# Min/Max of YearBuilt and YrSold
min_year_built = df_train['YearBuilt'].min()
max_year_built = df_train['YearBuilt'].max()
min_year_sold = df_train['YrSold'].min()
max_year_sold = df_train['YrSold'].max()

(min_year_built, max_year_built),(min_year_sold, max_year_sold)

((1872, 2010), (2006, 2010))

In [4]:
years_features = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']
years_data = df_train[years_features]
years_data.describe()

,YearBuilt,YearRemodAdd,GarageYrBlt,YrSold
count,1460.000000,1460.000000,1379.000000,1460.000000
mean,1971.267808,1984.865753,1978.506164,2007.815753
std,30.202904,20.645407,24.689725,1.328095
min,1872.000000,1950.000000,1900.000000,2006.000000
25%,1954.000000,1967.000000,1961.000000,2007.000000
50%,1973.000000,1994.000000,1980.000000,2008.000000
75%,2000.000000,2004.000000,2002.000000,2009.000000
max,2010.000000,2010.000000,2010.000000,2010.000000


In [5]:
years_data.head()

,YearBuilt,YearRemodAdd,GarageYrBlt,YrSold
0,2003,2003,2003.0,2008
1,1976,1976,1976.0,2007
2,2001,2002,2001.0,2008
3,1915,1970,1998.0,2006
4,2000,2000,2000.0,2008


## Building a Model - Decision Tree
* Select prediction target *y*
* Choose features (set of *X*)
* Build model
* Validate model using mean absolute error (MAE)
* Compare different tree sizes

In [6]:
y = df_train.SalePrice

feature_names = ['BedroomAbvGr', 'FullBath', 'LotArea']
X = df_train[feature_names]
X.describe()

,BedroomAbvGr,FullBath,LotArea
count,1460.000000,1460.000000,1460.000000
mean,2.866438,1.565068,10516.828082
std,0.815778,0.550916,9981.264932
min,0.000000,0.000000,1300.000000
25%,2.000000,1.000000,7553.500000
50%,3.000000,2.000000,9478.500000
75%,3.000000,2.000000,11601.500000
max,8.000000,3.000000,215245.000000


In [7]:
train_model = DecisionTreeRegressor(random_state = 1)
train_model.fit(X, y)

DecisionTreeRegressor(random_state=1)

In [8]:
print("Making predictions for the following 5 houses:")
print(X.head())
print("The predictions are:")
print(train_model.predict(X.head()))

Making predictions for the following 5 houses:
   BedroomAbvGr  FullBath  LotArea
0             3         2     8450
1             3         2     9600
2             3         2    11250
3             3         1     9550
4             4         2    14260
The predictions are:
[195625. 167125. 223500. 140000. 250000.]


In [9]:
print("Actual prices:")
print(df_train.SalePrice.head())

Actual prices:
0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64


In [10]:
feature_names = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = df_train[feature_names]
X.describe()

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,10516.828082,1971.267808,1162.626712,346.992466,1.565068,2.866438,6.517808
std,9981.264932,30.202904,386.587738,436.528436,0.550916,0.815778,1.625393
min,1300.000000,1872.000000,334.000000,0.000000,0.000000,0.000000,2.000000
25%,7553.500000,1954.000000,882.000000,0.000000,1.000000,2.000000,5.000000
50%,9478.500000,1973.000000,1087.000000,0.000000,2.000000,3.000000,6.000000
75%,11601.500000,2000.000000,1391.250000,728.000000,2.000000,3.000000,7.000000
max,215245.000000,2010.000000,4692.000000,2065.000000,3.000000,8.000000,14.000000


In [11]:
train_model.fit(X, y)

DecisionTreeRegressor(random_state=1)

In [12]:
print("Making predictions for the following 5 houses: ", X.head())
print("The predictions are: ", train_model.predict(X.head()))
print("Actual target values for those homes: ", y.head().tolist())

Making predictions for the following 5 houses:     LotArea  YearBuilt  1stFlrSF  2ndFlrSF  FullBath  BedroomAbvGr  \
0     8450       2003       856       854         2             3   
1     9600       1976      1262         0         2             3   
2    11250       2001       920       866         2             3   
3     9550       1915       961       756         1             3   
4    14260       2000      1145      1053         2             4   

   TotRmsAbvGrd  
0             8  
1             6  
2             6  
3             7  
4             9  
The predictions are:  [208500. 181500. 223500. 140000. 250000.]
Actual target values for those homes:  [208500, 181500, 223500, 140000, 250000]


In [13]:
# calcalate mean absolute error
predicted_home_prices = train_model.predict(X)
mean_absolute_error(y, predicted_home_prices)

62.35433789954339

In [14]:
# split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0)

train_model = DecisionTreeRegressor(random_state=1)
train_model.fit(train_X, train_y)

# make validation predictions
val_predictions = train_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

32966.449315068494


In [15]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=1)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

In [25]:
candidate_max_leaf_nodes = [5, 50, 500, 5000]

scores = {leaf_size: get_mae(leaf_size, train_X, val_X, train_y, val_y) for leaf_size in candidate_max_leaf_nodes}

best_tree_size = min(scores, key=scores.get)
scores

{5: 35190.33670788684,
 50: 27825.888386265695,
 500: 32345.501889541203,
 5000: 32677.51506849315}

In [21]:
# using best value for max_leaf_nodes
final_model = DecisionTreeRegressor(max_leaf_nodes=best_tree_size, random_state=1)
final_model.fit(X, y)

DecisionTreeRegressor(max_leaf_nodes=50, random_state=1)

## Building a Model - Random Forests
* Select prediction target *y*
* Choose features (set of *X*)
* Build model
* Validate model using mean absolute error (MAE)
* Compare different tree sizes

In [23]:
forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, preds))

23009.206570906717
